In [ ]:
import sys


import numpy as np
import os
import time
import tensorflow as tf

In [ ]:
#path_to_file = tf.keras.utils.get_file('01-Genesis.org', 'https://github.com/bc-abe/plain-text-bibles/raw/master/AKJV-org/01-Genesis.org')

path_to_file = tf.keras.utils.get_file('Bible.txt', 'https://github.com/ARSO-0/old_testament_sample_text/blob/main/Bible.txt?raw=true')

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it run on gpu
print(f'Length of text: {len(text)} characters')

# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

7373217/7373217 [==============================] - 0s 0us/step
Length of text: 4164321 characters
89 unique characters


In [ ]:

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


#Create training batches

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

print(dataset)


#Build The Model

# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
model.summary()




<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>
(64, 100, 90) # (batch_size, sequence_length, vocab_size)
Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  23040     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  92250     
                                                                 
Total params: 4,053,594
Trainable params: 4,053,594
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Train the model

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

history = model.fit(dataset, epochs=20, callbacks=[checkpoint_callback])


Epoch 1/20
644/644 [==============================] - 35s 51ms/step - loss: 2.1617
Epoch 2/20
644/644 [==============================] - 36s 54ms/step - loss: 1.4890
Epoch 3/20
644/644 [==============================] - 36s 54ms/step - loss: 1.3402
Epoch 4/20
644/644 [==============================] - 37s 55ms/step - loss: 1.2654
Epoch 5/20
644/644 [==============================] - 37s 56ms/step - loss: 1.2136
Epoch 6/20
644/644 [==============================] - 38s 57ms/step - loss: 1.1715
Epoch 7/20
644/644 [==============================] - 37s 55ms/step - loss: 1.1349
Epoch 8/20
644/644 [==============================] - 37s 56ms/step - loss: 1.1030
Epoch 9/20
644/644 [==============================] - 37s 56ms/step - loss: 1.0734
Epoch 10/20
644/644 [==============================] - 37s 56ms/step - loss: 1.0476
Epoch 11/20
644/644 [==============================] - 37s 56ms/step - loss: 1.0248
Epoch 12/20
644/644 [==============================] - 37s 56ms/step - loss: 1.0052
E

#Generate text

Each time you call the model you pass in some text and an internal state.
The model returns a prediction for the next character and its new state.
Pass the prediction and state back in to continue generating text.
The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
start = time.time()
states = None
next_char = tf.constant(['Бытие'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


Бытие: вень около трех дней во, даре пророка, Господа и Богом Рагава и Аарона и сыновья их, и сыновей в Иарине и в Шамафе, и вас; вы научились смиренномудрием огорченными людьми:
14 нужно истребление сходов приновятся в землю, которую Я даю тебе.
10 Смотри по жребии моем, который приноси Емугу свою и проклят тестью, собирается седому, сказав: "зайта ваши Царствия Божия: "разрушит слово, которое уже не будешь взять храм в комнатах твоих".
16 Итак похорони меча, но авно пройдет гонцев и прелюбодействие, согрешав, пока остается над нею во всесожжение Господу Богу вашему;
6 как же спасало предопределение, некому духовным дать от страны севира; подая им вперед.
16 Видите ли, Господи! что мне слово ко мне: не тотчас отнято от Отца во дни его с Иисусом, и оберет Иезении, сын Карея, и Сидону и братьям
2 и с ним в Иудее, воскликнул в своем [обещанию; постоянно оставался этим своим
28 и начал говорит им Иисус: разве нет там, как делал царь царство в внешнем доме, иду к Тебе, и назначил дел Его.
